<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS767/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler 

In [2]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)
def sigmoid(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = 1.0 if item > 0.0 else 0.0 
  return output
def relu(x):
  output = [0] * len(x)
  for index, item in enumerate(x): 
    output[index] = item if item > 0.0 else item * 0.05
  return output

In [3]:
def categorical_cross_entropy(actual, predicted):
	sum_score = 0.0
	for i in range(len(actual)):
			sum_score += actual[i] * math.log(1e-15 + predicted[i])
	mean_sum_score = (1.0 / len(actual)) * sum_score
	return -mean_sum_score

In [4]:
output_labels = [0.0, 1.0, 2.0]

In [5]:
'''
Class for the layer of the network 
which manages weights, bias, output and errors  
'''
class Layer():
  def __init__(self, name, units, input_dim):
    self.name = name
    self.w = [[random.uniform(0,1) for _ in range(input_dim)] for _ in range(units)]
    self.b = [0 for _ in range(units)]
    self.output = [bias for bias in self.b]
    self.next_layer = None
    self.previous_layer = None
    self.error = [0 for _ in range(units)]
  def set_previous_layer(self, previous_layer):
    self.previous_layer = previous_layer
  def set_next_layer(self, next_layer):
    self.next_layer = next_layer
  '''
  function to calculate the output of the neurons at current layer with 
  the given inputs
  '''
  def calculate_output(self, inputs, activation=sigmoid):
    self.output = [bias for bias in self.b]
    self.output += np.dot(self.w, inputs)
    self.output = activation(self.output)
    return self.output 
  '''
  function to calculate the error in the output at current layer with 
  the given expected output
  '''
  def update_error(self, expected_output):
    # last layer
    if self.next_layer == None:
      inputs = self.previous_layer.output
      # total_error = sum([(exp_output-self.output[i])**2 for i, exp_output in enumerate(expected_output)])
      # total_error = log_loss(expected_output, [self.output], labels=output_labels)
      total_error = categorical_cross_entropy(self.output, expected_output)
      for row_index, col in enumerate(self.w):
        self.error[row_index] = total_error * self.output[row_index] * (1-self.output[row_index]) 
    # layers other than last layer
    else:
      weights = self.next_layer.w
      next_layer_error = self.next_layer.error
      for output_idx, output in enumerate(self.output):
        self.error[output_idx] = sum([weight[output_idx] * next_layer_error[weight_idx] * output * (1-output) for weight_idx, weight in enumerate(weights)])
  '''
  function to update the weights at current layer with 
  the given inputs
  '''
  def update_weights(self, learning_rate, row):
    if self.next_layer == None:
      inputs = self.previous_layer.output
      for row_index, col in enumerate(self.w):
        for col_index, weight in enumerate(col):
          self.w[row_index][col_index] = self.w[row_index][col_index] + (learning_rate * self.error[row_index] * inputs[col_index])
          self.b[row_index] += learning_rate * self.error[row_index]
    else:
      inputs = row
      for row_index, col in enumerate(self.w):
        for col_index, weight in enumerate(col):
          self.w[row_index][col_index] = self.w[row_index][col_index] + (learning_rate * self.error[row_index] * inputs[col_index])
          self.b[row_index] += learning_rate * self.error[row_index]
  
    

In [6]:
# create network with the hidden and output layers
def reset_layers(inputs, hidden_layer, output):
  layer2 = Layer("Output", output, hidden_layer)
  layer1 = Layer("Hidden1", hidden_layer, inputs)
  layer2.set_previous_layer(layer1)
  layer1.set_next_layer(layer2)
  layers = [layer1, layer2]
  return layers

In [7]:
# Load IRIS data
iris = load_iris()

iris_columns = ['sepal_len', 'sepal_width', 'petal_length', 'petal_width', 'target']
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris_columns)

In [8]:
# select desired columns, split into training and test datasets
# standardize the inputs
from sklearn.model_selection import train_test_split
# X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
X = df[['sepal_width', 'petal_length']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)


scaler_training = StandardScaler() 
scaler_training.fit(X_train)
scaled_values_training = scaler_training.transform(X_train)
scaled_values_testing = scaler_training.transform(X_test)

X_train = pd.DataFrame(scaled_values_training, columns=[*X_train.columns.values])
X_test = pd.DataFrame(scaled_values_testing, columns=[*X_test.columns.values])

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [9]:
# set random seed
import random
random.seed(68)
selected_rows = random.sample(range(0,119),   5)

In [10]:
# label binarizer
lb = preprocessing.LabelBinarizer()
lb.fit([0,1,2])

LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False)

In [11]:
def train_weights(X_train, y_train, num_epochs, learning_rate, layers):
  index = 0;
  # select mini batch
  mini_batch = X_train.loc[selected_rows, :]
  y_train = y_train.loc[selected_rows, :]
  while index < num_epochs:
    sum_error = 0
    print("============================================")
    print("=================Iteration %d==============="%(index+1))
    print("============================================")
    for rowIndex, row in mini_batch.iterrows():
      # pass the inputs and get the outputs in a typical feed forward network.
      output = row
      for layer_idx, layer in enumerate(layers):
        if layer_idx == len(layers) - 1:
          output = layer.calculate_output(output, softmax)
        else:
          output = layer.calculate_output(output, relu)
      # calculate total error 
      expected_output = lb.transform([y_train.loc[rowIndex,'target']])[0]
      sum_error += sum([(exp_output-output[i])**2 for i, exp_output in enumerate(expected_output)])

      # update error at each layer
      for layer_idx, layer in enumerate(reversed(layers)):
        # layer.update_error([y_train.loc[rowIndex,'target']])
        layer.update_error(expected_output)
      print("error of output layer", layers[1].error)
      print("error of hidden layer", layers[0].error)  
      # update weights at each layer
      for layer_idx, layer in enumerate(reversed(layers)):
        layer.update_weights(learning_rate, row)
      
      # error = log_loss([y_train.loc[rowIndex,'target']], [output], labels=[0.0, 1.0, 2.0])
    print("Entropy based loss: %f"% (sum_error))
      # if error != 0:
      #   layer1_error = layers[1].update_weights(error, layers[0].output, learning_rate)
      #   layers[0].update_weights(layer1_error, row, learning_rate)
    print("Weights of output layer", layers[1].w)
    print("Weights of hidden layer", layers[0].w)
    index = index + 1

layers = reset_layers(2, 3, 3)
print("Creating model using training data")
train_weights(X_train, y_train, 2, 0.5, layers)

Creating model using training data
=================Iteration 1===============
error of output layer [1.8754086013465177, 1.7325333993731837, 1.6967238030474185]
error of hidden layer [0.6856222888517162, -0.012493294561555393, -0.009954467320011394]
error of output layer [2.2756176314800016, 1.6497474488384418, 1.4005201506001135]
error of hidden layer [-3.4923703141141234, -0.02939211258730893, -0.03218172820769665]
error of output layer [2.314178477136166, 1.7477016564433796, 1.3316065430229087]
error of hidden layer [-4.610978256062003, -0.167816876725929, -0.25876285207105754]
error of output layer [0.3190722804706468, 0.2472252961268001, 0.10024995213060675]
error of hidden layer [-0.41361074808804504, -0.0021621519762701903, -0.009002232890080663]
error of output layer [1.6159827265233397, 1.3057506109174999, 0.46992129041574987]
error of hidden layer [-7.100160394963258, -0.027906772416722545, -0.09505659564295987]
Entropy based loss: 4.281759
Weights of output layer [[2.044867

In [12]:
# select desired columns, split into training and test datasets
# standardize the inputs
from sklearn.model_selection import train_test_split
X = df[['sepal_len', 'sepal_width', 'petal_length', 'petal_width']]
y = df[['target']]

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state = 10)


scaler_training = StandardScaler() 
scaler_training.fit(X_train)
scaled_values_training = scaler_training.transform(X_train)
scaled_values_testing = scaler_training.transform(X_test)

X_train = pd.DataFrame(scaled_values_training, columns=[*X_train.columns.values])
X_test = pd.DataFrame(scaled_values_testing, columns=[*X_test.columns.values])

X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)


In [18]:
layers = reset_layers(4, 5, 3)
train_weights(X_train, y_train, 2, 0.5, layers)

=================Iteration 1===============
error of output layer [1.3883555655755018, 1.7167845311626557, 1.693694629492002]
error of hidden layer [0.3558778605162929, 0.8947744233284439, 0.567037412652011, -0.249267636271698, 0.35613520575305235]
error of output layer [0.31886539237147316, 1.2486251351941033, 1.301064636369684]
error of hidden layer [-0.8676264394919198, -17.730124931041296, -6.966835143813515, -0.18406737032136983, -0.6991921117608071]
error of output layer [1.66655991725264, 1.199625258894805, 0.6417351991456913]
error of hidden layer [-0.7314387798995976, -69.87555237656701, -11.084711989581113, -0.13667740375813575, -0.5107176212454484]
error of output layer [1.2431151005560965e-07, 1.238354444298128e-07, 4.761642098664742e-10]
error of hidden layer [-4.004215221574913e-08, -2.56040381980099e-06, -3.062077523862442e-07, -1.1164835061173562e-07, -2.8132275426099095e-08]
error of output layer [8.806739388136755e-12, 8.807400508123762e-12, 1.559789504775641e-17]
err